<a href="https://colab.research.google.com/github/MwangiMwaniki/Data-science/blob/main/Moringa_Data_Science_Prep_W4_Independent_Project_2021_08_ROY_MWANGI_Python_Notebookd1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview

Research Question

Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.
Bonus Questions (Optional)

What is the most popular hour for returning cars?

What station is the most popular?
Overall?

At the most popular picking hour?

What postal code is the most popular for picking up Blue cars?
Does the most popular station belong to that postal code?
Overall? At the most popular picking hour?

Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 

In [2]:
#Import libraries

import numpy as np
import pandas as pd

In [3]:
#Load datasets
AUTOdf = pd.read_csv('/content/Autolib_dataset.csv')
AUTOdf.head(10)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
5,8 Boulevard Voltaire,0,0,0,0,0,nonexistent,Paris,NaN,paris-voltaire-8,STATION,"48.8657658, 2.3664376",75011,Paris/Voltaire/8,operational,NaN,4,station,ok,nonexistent,2018,4,6,7,2
6,37 rue Leblanc,0,0,0,0,0,nonexistent,Paris,"Station en parking (niv -1), accÃ¨s 37 rue Leb...",paris-citroencevennes-parking,PARKING,"48.83874, 2.27641",75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,NaN,0,station,closed,nonexistent,2018,4,8,18,20
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,0,nonexistent,Nanterre,NaN,nanterre-luaps-17,STATION,"48.88069, 2.21063",92000,Nanterre/Luaps/17,operational,NaN,0,station,ok,nonexistent,2018,4,4,22,13
8,34 avenue Jean Moulin,1,1,0,0,0,nonexistent,Paris,NaN,paris-jeanmoulin-34,STATION,"48.8266807, 2.3237355",75014,Paris/Jean Moulin/34,operational,NaN,4,station,ok,nonexistent,2018,4,2,22,58
9,41 boulevard de Rochechouart,6,6,0,0,0,nonexistent,Paris,NaN,paris-anvers-parking,PARKING,"48.88267, 2.34405",75009,Paris/Anvers/Parking,operational,NaN,0,station,ok,nonexistent,2018,4,4,15,2


In [4]:
ALTdf = pd.read_csv('/content/Autolib_dataset.csv')
ALTdf.head(10)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
5,8 Boulevard Voltaire,0,0,0,0,0,nonexistent,Paris,NaN,paris-voltaire-8,STATION,"48.8657658, 2.3664376",75011,Paris/Voltaire/8,operational,NaN,4,station,ok,nonexistent,2018,4,6,7,2
6,37 rue Leblanc,0,0,0,0,0,nonexistent,Paris,"Station en parking (niv -1), accÃ¨s 37 rue Leb...",paris-citroencevennes-parking,PARKING,"48.83874, 2.27641",75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,NaN,0,station,closed,nonexistent,2018,4,8,18,20
7,17 Rue des Luaps ProlongÃ©e,3,3,1,0,0,nonexistent,Nanterre,NaN,nanterre-luaps-17,STATION,"48.88069, 2.21063",92000,Nanterre/Luaps/17,operational,NaN,0,station,ok,nonexistent,2018,4,4,22,13
8,34 avenue Jean Moulin,1,1,0,0,0,nonexistent,Paris,NaN,paris-jeanmoulin-34,STATION,"48.8266807, 2.3237355",75014,Paris/Jean Moulin/34,operational,NaN,4,station,ok,nonexistent,2018,4,2,22,58
9,41 boulevard de Rochechouart,6,6,0,0,0,nonexistent,Paris,NaN,paris-anvers-parking,PARKING,"48.88267, 2.34405",75009,Paris/Anvers/Parking,operational,NaN,0,station,ok,nonexistent,2018,4,4,15,2


CLEANING DATASET


In [ ]:
#The related "Scheduled at" column, by extension, isn't necessary. 
#and the comments column also happen to be the 2most incomplete columns:
AUTOdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

In [5]:
#Computing IQR for each column to detect Outliers

Q1 = AUTOdf.quantile(0.25)
Q3 = AUTOdf.quantile(0.75)
IQR = Q3 - Q1

print(IQR)

Cars                      4.0
Bluecar counter           4.0
Utilib counter            0.0
Utilib 1.4 counter        0.0
Charge Slots              0.0
Postal code           17308.0
Slots                     3.0
year                      0.0
month                     0.0
day                       4.0
hour                     12.0
minute                   30.0
dtype: float64


In [6]:
#Removing the outliers

print((AUTOdf  < (Q1 - 1.5 * IQR)) |(AUTOdf > (Q3 + 1.5 * IQR)))

AUTOdf2 = AUTOdf[~((AUTOdf < (Q1 - 1.5 * IQR)) |(AUTOdf > (Q3 + 1.5 * IQR))).any(axis=1)]
AUTOdf2.shape

      Address  Bluecar counter   Cars  ...  minute  month   year
0       False            False  False  ...   False  False  False
1       False            False  False  ...   False  False  False
2       False            False  False  ...   False  False  False
3       False            False  False  ...   False  False  False
4       False            False  False  ...   False  False  False
...       ...              ...    ...  ...     ...    ...    ...
4995    False            False  False  ...   False  False  False
4996    False            False  False  ...   False  False  False
4997    False            False  False  ...   False  False  False
4998    False            False  False  ...   False  False  False
4999    False            False  False  ...   False  False  False

[5000 rows x 25 columns]


(3317, 25)

In [15]:
#We first combine Date & Time in preparation for Analysis
#We also drop the unnecessary split date & time columns

#AUTOdf2['Date'] = AUTOdf2["year"].map(str) +':'+ AUTOdf2["month"].map(str) +':'+ AUTOdf2["day"].map(str)
#AUTOdf2.head()
#AUTOdf2['Time']=AUTOdf2["hour"].map(str) +':'+ AUTOdf2["minute"].map(str)
#AUTOdf2.head()
#AUTOdf2['Date.Time']= AUTOdf2["Date"].map(str) +':'+ AUTOdf2["Time"].map(str)
#AUTOdf2.head()
AUTOdf2.drop(['year','month','day','minute','Date','Time'],axis=1,inplace=True)
AUTOdf2.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,hour,Date.Time
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,11,2018:4:8:11:43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,7,2018:4:6:7:24
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,17,2018:4:8:17:23
5,8 Boulevard Voltaire,0,0,0,0,0,nonexistent,Paris,NaN,paris-voltaire-8,STATION,"48.8657658, 2.3664376",75011,Paris/Voltaire/8,operational,NaN,4,station,ok,nonexistent,7,2018:4:6:7:2
6,37 rue Leblanc,0,0,0,0,0,nonexistent,Paris,"Station en parking (niv -1), accÃ¨s 37 rue Leb...",paris-citroencevennes-parking,PARKING,"48.83874, 2.27641",75015,Paris/CitroÃ«n CÃ©vennes/Parking,broken,NaN,0,station,closed,nonexistent,18,2018:4:8:18:20


In [16]:
#Last action is to check for & deal with duplicate values (if any) in our DataFrame

AUTOdf2.duplicated().sum()

0

DATA ANALYSIS

In [17]:
#Bluecar activity for Paris in the month of April
AUTOdf3 = AUTOdf2[(AUTOdf2['City']=='Paris') & (AUTOdf2['Bluecar counter']!=0)]
AUTOdf3.head(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,hour,Date.Time
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,7,2018:4:6:7:24
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,17,2018:4:8:17:23
8,34 avenue Jean Moulin,1,1,0,0,0,nonexistent,Paris,NaN,paris-jeanmoulin-34,STATION,"48.8266807, 2.3237355",75014,Paris/Jean Moulin/34,operational,NaN,4,station,ok,nonexistent,22,2018:4:2:22:58
9,41 boulevard de Rochechouart,6,6,0,0,0,nonexistent,Paris,NaN,paris-anvers-parking,PARKING,"48.88267, 2.34405",75009,Paris/Anvers/Parking,operational,NaN,0,station,ok,nonexistent,15,2018:4:4:15:2
17,182 Boulevard Voltaire,6,6,0,0,0,operational,Paris,NaN,paris-voltaire-182,STATION,"48.8548, 2.385047",75011,Paris/Voltaire/182,operational,NaN,0,station,ok,nonexistent,12,2018:4:9:12:14


In [37]:
#Grouping data by station

AUTOdf4 = AUTOdf3.sort_values(['Date.Time']).groupby(['ID','Date.Time']).sum()
AUTOdf4.head(10)

Cars  Bluecar counter  ...  Slots  hour
ID                   Date.Time                              ...             
paris-25aout1944-8   2018:4:1:18:40     1                1  ...      2    18
                     2018:4:2:12:49     2                2  ...      1    12
                     2018:4:2:7:37      4                4  ...      0     7
                     2018:4:4:18:48     1                1  ...      1    18
                     2018:4:4:3:10      3                3  ...      0     3
paris-adolphemax-6   2018:4:2:11:1      4                4  ...      0    11
paris-adolphemille-8 2018:4:1:1:53      1                1  ...      1     1
                     2018:4:3:12:25     2                2  ...      1    12
                     2018:4:3:21:53     4                4  ...      0    21
paris-adolpheyvon-6  2018:4:2:14:15     5                5  ...      0    14

[10 rows x 8 columns]

In [23]:
#function to indicate car pickup and  return.( +ve = pickup, -ve = return )

AUTOdf4['return_or_pick'] = AUTOdf4.groupby(['ID'])['Bluecar counter'].apply(lambda x:  x.shift(1) - x)
AUTOdf4['return_or_pick']
AUTOdf4.head(5)

Cars  ...  return_or_pick
ID                 Date.Time             ...                
paris-25aout1944-8 2018:4:1:18:40     1  ...             NaN
                   2018:4:2:12:49     2  ...            -1.0
                   2018:4:2:7:37      4  ...            -2.0
                   2018:4:4:18:48     1  ...             3.0
                   2018:4:4:3:10      3  ...            -2.0

[5 rows x 10 columns]

RESEARCH QUESTIONS

In [ ]:
#Identify the most popular hour of the day for picking up a shared electric car (Bluecar) 
#in the city of Paris over the month of April 2018.
# I select city as Paris
ALTdf =(columns={'Bluecar counter'})
Blu_Paris = ALTdf[(ALTdf.City == 'Paris').groupby(['hour']).sum()
Blu_Paris.sort_values(ascending = False).head(1)

In [26]:
#Most popular hour for returning cars

AUTOdf4[(AUTOdf4['rt_or_pick'] < 0)].groupby(['hour'])['rt_or_pick'].sum().sort_values(ascending = False).head(1)

hour
23   -5.0
Name: rt_or_pick, dtype: float64

In [27]:
#Most popular hour for picking cars

AUTOdf4[(AUTOdf4['rt_or_pick'] > 0)].groupby(['hour'])['rt_or_pick'].sum().sort_values(ascending = False).head(1)

hour
21    31.0
Name: rt_or_pick, dtype: float64

In [28]:
#Overall most popular picking station

AUTOdf4['rt_or_pick']=AUTOdf4['rt_or_pick'].abs()
AUTOdf4.head()
AUTOdf4.groupby('ID')['rt_or_pick'].sum().sort_values(ascending=False).head(1)

ID
paris-raymondpitet-7    11.0
Name: rt_or_pick, dtype: float64

In [31]:
#Overall most popular picking station at the most popular picking hour?

AUTOdf4[(AUTOdf4['rt_or_pick'] > 0) & (AUTOdf4['hour']==21)].groupby(['hour','ID'])['rt_or_pick'].sum().sort_values(ascending = False).head(1)

hour  ID                 
21    paris-saintmande-94    4.0
Name: rt_or_pick, dtype: float64

In [32]:
#Most popular postcode for picking blue cars
AUTOdf4[(AUTOdf4['rt_or_pick'] > 0)].groupby(['ID','Postal code'])['rt_or_pick'].sum().sort_values(ascending = False).head(1)

ID                    Postal code
paris-raymondpitet-7  75017          11.0
Name: rt_or_pick, dtype: float64

Considering Utilib and Utilib 1.4

In [33]:
AUTOdf5 = AUTOdf4
AUTOdf5.head(5)

Cars  ...  return_or_pick
ID                 Date.Time             ...                
paris-25aout1944-8 2018:4:1:18:40     1  ...             NaN
                   2018:4:2:12:49     2  ...            -1.0
                   2018:4:2:7:37      4  ...            -2.0
                   2018:4:4:18:48     1  ...             3.0
                   2018:4:4:3:10      3  ...            -2.0

[5 rows x 10 columns]

In [34]:
AUTOdf5['rt_or_pick'] = AUTOdf5.groupby(['ID'])['Utilib counter'].apply(lambda x:  x.shift(1) - x)
AUTOdf5['rt_or_pick']
AUTOdf5.head(7)

Cars  ...  return_or_pick
ID                   Date.Time             ...                
paris-25aout1944-8   2018:4:1:18:40     1  ...             NaN
                     2018:4:2:12:49     2  ...            -1.0
                     2018:4:2:7:37      4  ...            -2.0
                     2018:4:4:18:48     1  ...             3.0
                     2018:4:4:3:10      3  ...            -2.0
paris-adolphemax-6   2018:4:2:11:1      4  ...             NaN
paris-adolphemille-8 2018:4:1:1:53      1  ...             NaN

[7 rows x 10 columns]

In [35]:
AUTOdf6 = AUTOdf4
AUTOdf6.head(5)

Cars  ...  return_or_pick
ID                 Date.Time             ...                
paris-25aout1944-8 2018:4:1:18:40     1  ...             NaN
                   2018:4:2:12:49     2  ...            -1.0
                   2018:4:2:7:37      4  ...            -2.0
                   2018:4:4:18:48     1  ...             3.0
                   2018:4:4:3:10      3  ...            -2.0

[5 rows x 10 columns]

In [36]:
AUTOdf6['rt_or_pick'] = AUTOdf6.groupby(['ID'])['Utilib 1.4 counter'].apply(lambda x:  x.shift(1) - x)
AUTOdf6['rt_or_pick']
AUTOdf6.head(7)

Cars  ...  return_or_pick
ID                   Date.Time             ...                
paris-25aout1944-8   2018:4:1:18:40     1  ...             NaN
                     2018:4:2:12:49     2  ...            -1.0
                     2018:4:2:7:37      4  ...            -2.0
                     2018:4:4:18:48     1  ...             3.0
                     2018:4:4:3:10      3  ...            -2.0
paris-adolphemax-6   2018:4:2:11:1      4  ...             NaN
paris-adolphemille-8 2018:4:1:1:53      1  ...             NaN

[7 rows x 10 columns]